### Packages

In [6]:
!pip install arch

In [7]:
!pip install pmdarima

In [8]:
!pip install yfinance

In [9]:
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from statsmodels.tsa.arima_model import ARIMA
from arch import arch_model
import yfinance
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
sns.set()

### Loading the data

In [13]:
raw_data = yfinance.download (tickers = "^GSPC ^FTSE ^N225 ^GDAXI", start = "1994-01-07", end = "2020-03-20", interval = "1d", group_by = 'ticker', auto_adjust = True, threads = True)

[*********************100%%**********************]  4 of 4 completed


In [14]:
df_comp = raw_data.copy()

In [15]:
df_comp.head()

Ticker            ^FTSE                                                \
Price              Open         High          Low        Close Volume   
Date                                                                    
1994-01-07  3401.399902  3446.800049  3398.699951  3446.000000    0.0   
1994-01-10  3465.699951  3468.100098  3430.000000  3440.600098    0.0   
1994-01-11  3442.500000  3442.500000  3413.500000  3413.800049    0.0   
1994-01-12  3394.800049  3402.399902  3372.000000  3372.000000    0.0   
1994-01-13  3380.699951  3383.300049  3356.899902  3360.000000    0.0   

Ticker           ^GSPC                                                   \
Price             Open        High         Low       Close       Volume   
Date                                                                      
1994-01-07  467.089996  470.260010  467.029999  469.899994  324920000.0   
1994-01-10  469.899994  475.269989  469.549988  475.269989  319490000.0   
1994-01-11  475.269989  475.279999  473.269989  474.130005  305490000.0   
1994-01-12  474.130005  475.059998  472.140015  474.170013  310690000.0   
1994-01-13  474.170013  474.170013  471.799988  472.470001  277970000.0   

Ticker           ^GDAXI                                                \
Price              Open         High          Low        Close Volume   
Date                                                                    
1994-01-07  2218.959961  2227.639893  2201.820068  2224.949951    0.0   
1994-01-10  2231.840088  2238.010010  2222.000000  2225.000000    0.0   
1994-01-11  2225.429932  2235.610107  2225.179932  2228.100098    0.0   
1994-01-12  2227.120117  2227.790039  2182.060059  2182.060059    0.0   
1994-01-13  2171.500000  2183.709961  2134.100098  2142.370117    0.0   

Ticker             ^N225                                                   
Price               Open          High           Low         Close Volume  
Date                                                                       
1994-01-07  17842.980469  18131.410156  17787.480469  18124.009766    0.0  
1994-01-10  18186.519531  18567.060547  18186.519531  18443.439453    0.0  
1994-01-11  18481.849609  18671.669922  18373.039062  18485.250000    0.0  
1994-01-12  18447.339844  18807.080078  18301.929688  18793.880859    0.0  
1994-01-13  18770.380859  18823.380859  18548.750000  18577.259766    0.0

In [16]:
df_comp['spx'] = df_comp['^GSPC'].Close[:]
df_comp['dax'] = df_comp['^GDAXI'].Close[:]
df_comp['ftse'] = df_comp['^FTSE'].Close[:]
df_comp['nikkei'] = df_comp['^N225'].Close[:]

In [17]:
df_comp.head()

Ticker            ^FTSE                                                \
Price              Open         High          Low        Close Volume   
Date                                                                    
1994-01-07  3401.399902  3446.800049  3398.699951  3446.000000    0.0   
1994-01-10  3465.699951  3468.100098  3430.000000  3440.600098    0.0   
1994-01-11  3442.500000  3442.500000  3413.500000  3413.800049    0.0   
1994-01-12  3394.800049  3402.399902  3372.000000  3372.000000    0.0   
1994-01-13  3380.699951  3383.300049  3356.899902  3360.000000    0.0   

Ticker           ^GSPC                                                   ...  \
Price             Open        High         Low       Close       Volume  ...   
Date                                                                     ...   
1994-01-07  467.089996  470.260010  467.029999  469.899994  324920000.0  ...   
1994-01-10  469.899994  475.269989  469.549988  475.269989  319490000.0  ...   
1994-01-11  475.269989  475.279999  473.269989  474.130005  305490000.0  ...   
1994-01-12  474.130005  475.059998  472.140015  474.170013  310690000.0  ...   
1994-01-13  474.170013  474.170013  471.799988  472.470001  277970000.0  ...   

Ticker     ^GDAXI         ^N225                                            \
Price      Volume          Open          High           Low         Close   
Date                                                                        
1994-01-07    0.0  17842.980469  18131.410156  17787.480469  18124.009766   
1994-01-10    0.0  18186.519531  18567.060547  18186.519531  18443.439453   
1994-01-11    0.0  18481.849609  18671.669922  18373.039062  18485.250000   
1994-01-12    0.0  18447.339844  18807.080078  18301.929688  18793.880859   
1994-01-13    0.0  18770.380859  18823.380859  18548.750000  18577.259766   

Ticker                    spx          dax         ftse        nikkei  
Price      Volume                                                      
Date                                                                   
1994-01-07    0.0  469.899994  2224.949951  3446.000000  18124.009766  
1994-01-10    0.0  475.269989  2225.000000  3440.600098  18443.439453  
1994-01-11    0.0  474.130005  2228.100098  3413.800049  18485.250000  
1994-01-12    0.0  474.170013  2182.060059  3372.000000  18793.880859  
1994-01-13    0.0  472.470001  2142.370117  3360.000000  18577.259766  

[5 rows x 24 columns]

In [18]:
df_comp.tail()

Ticker            ^FTSE                                                       \
Price              Open         High          Low        Close        Volume   
Date                                                                           
2020-03-13  5237.500000  5696.500000  5237.500000  5366.100098  1.935105e+09   
2020-03-16  5366.100098  5366.100098  4898.799805  5151.100098  2.111568e+09   
2020-03-17  5151.100098  5309.000000  4978.799805  5294.899902  1.941165e+09   
2020-03-18  5294.899902  5294.899902  5006.200195  5080.600098  1.837075e+09   
2020-03-19  5080.600098  5181.000000  4942.399902  5151.600098  1.963412e+09   

Ticker            ^GSPC                                                       \
Price              Open         High          Low        Close        Volume   
Date                                                                           
2020-03-13  2569.989990  2711.330078  2492.370117  2711.020020  8.299070e+09   
2020-03-16  2508.590088  2562.979980  2380.939941  2386.129883  7.805450e+09   
2020-03-17  2425.659912  2553.929932  2367.040039  2529.189941  8.370250e+09   
2020-03-18  2436.500000  2453.570068  2280.520020  2398.100098  8.799300e+09   
2020-03-19  2393.479980  2466.969971  2319.780029  2409.389893  7.956100e+09   

Ticker      ...       ^GDAXI         ^N225                              \
Price       ...       Volume          Open          High           Low   
Date        ...                                                          
2020-03-13  ...  325900900.0  18183.470703  18184.460938  16690.599609   
2020-03-16  ...  302202400.0  17586.080078  17785.759766  16914.449219   
2020-03-17  ...  220092600.0  16726.949219  17557.039062  16378.940430   
2020-03-18  ...  207558900.0  17154.080078  17396.839844  16698.460938   
2020-03-19  ...  205539300.0  16995.769531  17160.970703  16358.190430   

Ticker                                         spx          dax         ftse  \
Price              Close       Volume                                          
Date                                                                           
2020-03-13  17431.050781  233400000.0  2711.020020  9232.080078  5366.100098   
2020-03-16  17002.039062  158100000.0  2386.129883  8742.250000  5151.100098   
2020-03-17  17011.529297  198800000.0  2529.189941  8939.099609  5294.899902   
2020-03-18  16726.550781  177200000.0  2398.100098  8441.709961  5080.600098   
2020-03-19  16552.830078  198900000.0  2409.389893  8610.429688  5151.600098   

Ticker            nikkei  
Price                     
Date                      
2020-03-13  17431.050781  
2020-03-16  17002.039062  
2020-03-17  17011.529297  
2020-03-18  16726.550781  
2020-03-19  16552.830078  

[5 rows x 24 columns]

In [19]:
del df_comp['^N225']
del df_comp['^GSPC']
del df_comp['^GDAXI']
del df_comp['^FTSE']
df_comp=df_comp.asfreq('b')
df_comp=df_comp.fillna(method='ffill')

In [20]:
df_comp.head()

Ticker,spx,dax,ftse,nikkei
Price,,,,
Date,,,,
1994-01-07,469.899994,2224.949951,3446.000000,18124.009766
1994-01-10,475.269989,2225.000000,3440.600098,18443.439453
1994-01-11,474.130005,2228.100098,3413.800049,18485.250000
1994-01-12,474.170013,2182.060059,3372.000000,18793.880859
1994-01-13,472.470001,2142.370117,3360.000000,18577.259766


### Creating Returns

In [21]:
df_comp['ret_spx'] = df_comp.spx.pct_change(1)*100
df_comp['ret_ftse'] = df_comp.ftse.pct_change(1)*100
df_comp['ret_dax'] = df_comp.dax.pct_change(1)*100
df_comp['ret_nikkei'] = df_comp.nikkei.pct_change(1)*100

### Splitting the Data

In [22]:
size = int(len(df_comp)*0.8)
df, df_test = df_comp.iloc[:size], df_comp.iloc[size:]

### Fitting a Model

In [23]:
from pmdarima.arima import auto_arima

In [24]:
model_auto = auto_arima(df.ret_ftse[1:])

In [25]:
model_auto

ARIMA(order=(4, 0, 5), scoring_args={}, suppress_warnings=True)

In [26]:
model_auto.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 5467
Model:               SARIMAX(4, 0, 5)   Log Likelihood               -8450.891
Date:                Wed, 13 Mar 2024   AIC                          16923.782
Time:                        21:05:40   BIC                          16996.453
Sample:                    01-10-1994   HQIC                         16949.137
                         - 12-23-2014                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0292      0.023      1.271      0.204      -0.016       0.074
ar.L1          0.0166      0.080      0.207      0.836      -0.141       0.174
ar.L2         -0.6505      0.077     -8.433      0.000      -0.802      -0.499
ar.L3         -0.1678      0.071     -2.360      0.018      -0.307      -0.028
ar.L4          0.2122      0.075      2.838      0.005       0.066       0.359
ma.L1         -0.0395      0.080     -0.493      0.622      -0.196       0.118
ma.L2          0.6050      0.078      7.766      0.000       0.452       0.758
ma.L3          0.0743      0.069      1.077      0.282      -0.061       0.210
ma.L4         -0.2075      0.074     -2.814      0.005      -0.352      -0.063
ma.L5         -0.0995      0.009    -11.125      0.000      -0.117      -0.082
sigma2         1.2885      0.013     96.355      0.000       1.262       1.315
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):              7482.37
Prob(Q):                              0.99   Prob(JB):                         0.00
Heteroskedasticity (H):               1.81   Skew:                            -0.19
Prob(H) (two-sided):                  0.00   Kurtosis:                         8.72
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

### Important Arguments

In [29]:
model_auto2 = auto_arima(df_comp.ret_ftse[1:], exogenous = df_comp[['ret_spx', 'ret_dax', 'ret_nikkei']][1:], m = 5,
                       max_order = None, max_p = 7, max_q = 7, max_d = 2, max_P = 4, max_Q = 4, max_D = 2,
                       maxiter = 50, alpha = 0.05, n_jobs = -1, trend = 'ct', information_criterion = 'bic',
                       out_of_sample_size = int(len(df_comp)*0.2))


# exogenous -> outside factors (e.g other time series)
# m -> seasonal cycle length
# max_order -> maximum amount of variables to be used in the regression (p + q)
# max_p -> maximum AR components
# max_q -> maximum MA components
# max_d -> maximum Integrations
# maxiter -> maximum iterations we're giving the model to converge the coefficients (becomes harder as the order increases)
# alpha -> level of significance, default is 5%, which we should be using most of the time
# n_jobs -> how many models to fit at a time (-1 indicates "as many as possible")
# trend -> "ct" usually
# information_criterion -> 'aic', 'aicc', 'bic', 'hqic', 'oob' 
#        (Akaike Information Criterion, Corrected Akaike Information Criterion,
#        Bayesian Information Criterion, Hannan-Quinn Information Criterion, or
#        "out of bag"--for validation scoring--respectively)
# out_of_smaple_size -> validates the model selection (pass the entire dataset, and set 20% to be the out_of_sample_size)

In [30]:
model_auto2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 SARIMAX Results                                 
=================================================================================
Dep. Variable:                         y   No. Observations:                 6834
Model:             SARIMAX(0, 0, [1], 5)   Log Likelihood              -10414.512
Date:                   Wed, 13 Mar 2024   AIC                          20837.024
Time:                           21:51:52   BIC                          20864.343
Sample:                                0   HQIC                         20846.449
                                  - 6834                                         
Covariance Type:                     opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0244      0.030      0.803      0.422      -0.035       0.084
drift      -4.616e-06      8e-06     -0.577      0.564   -2.03e-05    1.11e-05
ma.S.L5       -0.0508      0.007     -7.442      0.000      -0.064      -0.037
sigma2         1.3111      0.011    118.989      0.000       1.290       1.333
===================================================================================
Ljung-Box (L1) (Q):                   1.77   Jarque-Bera (JB):             16717.69
Prob(Q):                              0.18   Prob(JB):                         0.00
Heteroskedasticity (H):               0.76   Skew:                            -0.27
Prob(H) (two-sided):                  0.00   Kurtosis:                        10.64
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""